In [1]:
import os
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Evaluator, Pusher, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import tensorflow as tf
import pandas as pd
from tfx.proto import trainer_pb2, pusher_pb2 
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing
import tensorflow_model_analysis as tfma

# Set Variable

In [2]:
DATA_ROOT = r'data'
PIPELINE_NAME = r'Bima Prastyaji-pipeline'
SCHEMA_PIPELINE_NAME = r'spam-tfdv-schema'

# Output directory to store artifacts generated from TFX pipeline
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

# # Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join(PIPELINE_NAME, 'metadata.db')

# Output directory where created models from the pipeline will be exported
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

# Data Ingestion

In [3]:
# check the data
df = pd.read_csv('email_classification.csv')
df.head(5)

,email,label
0,Upgrade to our premium plan for exclusive acce...,ham
1,Happy holidays from our team! Wishing you joy ...,ham
2,We're hiring! Check out our career opportuniti...,ham
3,Your Amazon account has been locked. Click her...,spam
4,Your opinion matters! Take our survey and help...,ham


value pada feature label merupakan string sehingga harus diubah menjadi integer dengan cara encode menjadi ham : 0 & spam : 1

In [4]:
# encode the label
df.label = df.label.replace({'ham': 0, 'spam': 1})

# save the data
df.to_csv(os.path.join(DATA_ROOT, 'new_spam.csv'), index=False)

### Create CsvExampleGen component

Load data csv ke dalam pipeline

In [5]:
output = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=7),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=3)
    ])
)
example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [6]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Data Validation

### Create StatisticsGen component

In [7]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [8]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Create SchemaGen component

In [9]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [10]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'email',BYTES,required,,-
'label',INT,required,,-


### Create ExampleValidator component

membuat validator component untuk mendeteksi adanya anomali pada data

In [11]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema']
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [12]:
interactive_context.show(example_validator.outputs['anomalies'])

# Data Processing
Processing yang dilakukan adalah menghapus tanda baca dan mengubahnya menjadi huruf kecil pada feature email, dan juga melakukan on hot encoding pada label dengan menggunakan *compute_and_apply_vocabulary()*

### Create Processing Module

In [13]:
TRANSFORM_MODULE_FILE = r'spam_transform.py'

In [14]:
%%writefile {TRANSFORM_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft

LABEL_KEY = "label"
FEATURE_KEY = "email"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def preprocessing_fn(inputs):
    """
    Preprocess input features into transformed features
    
    Args:
        inputs: map from feature keys to raw features.
    
    Return:
        outputs: map from feature keys to transformed features.    
    """
    
    outputs = {}
    
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
                
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    
    return outputs


Writing spam_transform.py


In [15]:
# transform 
transform  = Transform(
    examples=example_gen.outputs['examples'],
    schema= schema_gen.outputs['schema'],
    module_file=os.path.abspath(TRANSFORM_MODULE_FILE)
)
interactive_context.run(transform)

INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\ed674ce74ec645ceb18153ad60fd4715\assets


INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Transform\transform_graph\5\.temp_path\tftransform_tmp\ed674ce74ec645ceb18153ad60fd4715\assets


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:struct2tensor is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


...

INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_decision_forests is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

# Training and Evaluation

### Create Tuner for Hyperparameter tuning

In [16]:
TUNER_MODULE_FILE = r'spam_tuner.py'

In [17]:
%%writefile {TUNER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from typing import NamedTuple, Dict, Text, Any
from keras_tuner.engine import base_tuner
from tensorflow.keras import layers
import keras_tuner as kt

LABEL_KEY = "label"
FEATURE_KEY = "email"

TunerFnResult = NamedTuple("TunerFnResult", [
    ("tuner", base_tuner.BaseTuner),
    ("fit_kwargs", Dict[Text, Any]),
])

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="val_binary_accuracy",
    mode="max",
    verbose=1,
    patience=2,
    baseline=1.0
)

def transformed_name(key):
    """Rename transformed key"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=32) -> tf.data.Dataset:
    """Get post_transform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # Create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key=transformed_name(LABEL_KEY))
    return dataset

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

def model_builder(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), dtype=tf.string, name=transformed_name(FEATURE_KEY)),
        tf.keras.layers.Lambda(lambda x: tf.reshape(x, [-1])),
        vectorize_layer,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=hp.Int("embed_dims", min_value=16, max_value=64, step=32), name="embedding"),
        tf.keras.layers.LSTM(units=hp.Int('lstm_units',min_value=32, max_value=64, step=32), return_sequences=True),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(units=hp.Int('dense_units', min_value=32, max_value=64, step=32), activation='relu'),
        tf.keras.layers.Dropout(rate=hp.Float('drop_rate', min_value=0.1, max_value=0.5, step=0.1)),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    tf.keras.backend.clear_session()
    
    model.compile(
        loss='binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    )
    
    return model

def tuner_fn(fn_args):
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    train_set = input_fn(fn_args.train_files, tf_transform_output, num_epochs=10)
    eval_set = input_fn(fn_args.eval_files, tf_transform_output, num_epochs=10)
    
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    tuner = kt.RandomSearch(
        model_builder,
        objective='val_binary_accuracy',
        max_trials=20,
        directory=fn_args.working_dir,
        project_name='spam_classification'
    )
    
    return TunerFnResult(
        tuner=tuner,
        fit_kwargs={
            "callbacks": [early_stopping_callback],
            "x": train_set,
            "validation_data": eval_set,
            "steps_per_epoch": fn_args.train_steps,
            "validation_steps": fn_args.eval_steps,
            "epochs": 10
        },
    )


Writing spam_tuner.py


In [18]:
tuner = Tuner(
    module_file=TUNER_MODULE_FILE,
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']), 
)
interactive_context.run(tuner)

Trial 20 Complete [00h 00m 07s]
val_binary_accuracy: 0.9523809552192688

Best val_binary_accuracy So Far: 1.0
Total elapsed time: 00h 03m 00s
Results summary
Results in pipelines\Bima Prastyaji-pipeline\.temp\6\spam_classification
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 02 summary
Hyperparameters:
embed_dims: 16
lstm_units: 64
dense_units: 64
drop_rate: 0.4
learning_rate: 0.001
Score: 1.0

Trial 07 summary
Hyperparameters:
embed_dims: 16
lstm_units: 32
dense_units: 64
drop_rate: 0.2
learning_rate: 0.01
Score: 1.0

Trial 08 summary
Hyperparameters:
embed_dims: 48
lstm_units: 64
dense_units: 32
drop_rate: 0.5
learning_rate: 0.001
Score: 1.0

Trial 11 summary
Hyperparameters:
embed_dims: 48
lstm_units: 32
dense_units: 32
drop_rate: 0.1
learning_rate: 0.01
Score: 1.0

Trial 13 summary
Hyperparameters:
embed_dims: 16
lstm_units: 64
dense_units: 64
drop_rate: 0.30000000000000004
learning_rate: 0.01
Score: 1.0

Trial 03 summary
Hyperparameters:
emb

ExecutionResult(
    component_id: Tuner
    execution_id: 6
    outputs:
        best_hyperparameters: OutputChannel(artifact_type=HyperParameters, producer_component_id=Tuner, output_key=best_hyperparameters, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        tuner_results: OutputChannel(artifact_type=TunerResults, producer_component_id=Tuner, output_key=tuner_results, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

### Create Trainer Module

In [19]:
TRAINER_MODULE_FILE = r'spam_trainer.py'

In [20]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import json
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "label"
FEATURE_KEY = "email"


def transformed_name(key):
    """Rename transformed key"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Small utility returning a record reader that can read gzip'ed files"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def input_fn(file_pattern, 
             tf_transform_output,
             num_epochs,
             batch_size=32)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""
    
    # Get post_transform feature spec
    transform_feature_spec = (tf_transform_output.transformed_feature_spec().copy())
    
    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset
 
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100
 
vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)
 
 
embedding_dim=16
def model_builder(hp):
    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(1,), dtype=tf.string, name=transformed_name(FEATURE_KEY)),
        tf.keras.layers.Lambda(lambda x: tf.reshape(x, [-1])),
        vectorize_layer,
        tf.keras.layers.Embedding(input_dim=VOCAB_SIZE, output_dim=hp['embed_dims'], name="embedding"),
        tf.keras.layers.LSTM(hp['lstm_units'], return_sequences=True),
        tf.keras.layers.GlobalMaxPooling1D(),
        tf.keras.layers.Dense(hp['dense_units'], activation='relu'),
        tf.keras.layers.Dropout(hp['drop_rate']),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    
    # inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    # reshaped_narrative = tf.reshape(inputs, [-1])
    # x = vectorize_layer(reshaped_narrative)
    # x = layers.Embedding(input_dim=VOCAB_SIZE,output_dim=embedding_dim, name="embedding")(x)
    # x = tf.keras.layers.LSTM(64, return_sequences=True)(x)
    # x = layers.GlobalMaxPooling1D()(x)
    # x = layers.Dense(64, activation='relu')(x)
    # x = layers.Dense(32, activation="relu")(x)
    # outputs = layers.Dense(1, activation='sigmoid')(x)
    
    # model = tf.keras.Model(inputs=inputs, outputs = outputs)
    
    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp['learning_rate']),
        metrics=[tf.keras.metrics.BinaryAccuracy()]
    
    )
    
    # print(model)
    model.summary()
    return model 
 
 
def _get_serve_tf_examples_fn(model, tf_transform_output):
    
    model.tft_layer = tf_transform_output.transform_features_layer()
    
    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        
        feature_spec = tf_transform_output.raw_feature_spec()
        
        feature_spec.pop(LABEL_KEY)
        
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        
        transformed_features = model.tft_layer(parsed_features)
        
        # get predictions using the transformed features
        return model(transformed_features)
        
    return serve_tf_examples_fn
    
def run_fn(fn_args: FnArgs) -> None:
    hp = fn_args.hyperparameters["values"]
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')
    
    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )
    
    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=2)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)
    
    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)
    
    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])
    
    # Build the model
    model = model_builder(hp)
    
    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)


Writing spam_trainer.py


In [21]:
trainer  = Trainer(
    module_file=os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    hyperparameters=tuner.outputs['best_hyperparameters'],
    train_args=trainer_pb2.TrainArgs(splits=['train']),
    eval_args=trainer_pb2.EvalArgs(splits=['eval']),
)
interactive_context.run(trainer)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lambda (Lambda)             (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 lstm (LSTM)                 (None, 100, 64)           20736     
                                                                 
 global_max_pooling1d (Glob  (None, 64)                0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 64)                4

INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


43/43 [==============================] - 8s 141ms/step - loss: 0.5666 - binary_accuracy: 0.7350 - val_loss: 0.4103 - val_binary_accuracy: 0.8571
Epoch 2/10
42/43 [============================>.] - ETA: 0s - loss: 0.2051 - binary_accuracy: 0.9472
Epoch 2: val_binary_accuracy improved from 0.85714 to 0.92857, saving model to pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


43/43 [==============================] - 8s 196ms/step - loss: 0.2040 - binary_accuracy: 0.9474 - val_loss: 0.3076 - val_binary_accuracy: 0.9286
Epoch 3/10
43/43 [==============================] - ETA: 0s - loss: 0.0860 - binary_accuracy: 0.9854
Epoch 3: val_binary_accuracy improved from 0.92857 to 0.95238, saving model to pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving
INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


...

INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:tensorflow_text is not available.


INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


INFO:tensorflow:Assets written to: pipelines\Bima Prastyaji-pipeline\Trainer\model\7\Format-Serving\assets


ExecutionResult(
    component_id: Trainer
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [22]:
model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 8
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [23]:
eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]

)

In [24]:
evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 9
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [25]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

FairnessIndicatorViewer(slicingMetrics=[{'sliceValue': 'Overall', 'slice': 'Overall', 'metrics': {'binary_accu…

# Create Pusher

In [26]:
pusher = Pusher(
model=trainer.outputs['model'],
model_blessing=evaluator.outputs['blessing'],
push_destination=pusher_pb2.PushDestination(
    filesystem=pusher_pb2.PushDestination.Filesystem(
        base_directory='serving_model_dir/spam-detection-model'))
    
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 10
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [31]:
# memastikan versi model yang digunakan tf-serving
import requests
from pprint import PrettyPrinter
    
pp = PrettyPrinter()
pp.pprint(requests.get("http://localhost:10001/v1/models/spam-detection-model").json())

{'model_version_status': [{'state': 'AVAILABLE',
                           'status': {'error_code': 'OK', 'error_message': ''},
                           'version': '1719479518'}]}
